# Outils de création d'une carte avec la création de plusieurs parcours.
Fichier nécessaire: un fichier en format .txt contenant la liste des noms de lieux à recenser sur la carte.

Les outils de recherches d'informations permettent de récupérer les emplacements de chaque lieu (coordonnées GPS), le type de lieu dans lequel il est catégorisé ainsi qu'une brève description du lieu. Un fichier sous la forme d'un tableau lisible avec Excel est créé avec les informations récoltées.

La récupération des coordonnées GPS permet de retracer les parcours.

La carte est enregistrée en format html.

Il est possible à la fin d'ajouter, à la fin du notebook, son propre fond de carte.

Exemple d'utilisation: deux chemin cités dans deux chapitres différentes d'un ouvrage. 

**Avant de lancer le notebook**:

Créer un dossier sur son Google Drive intitulé "Creation-de-cartes".
Placer dans ce dossier ce notebook, ainsi que le fichier .txt contenant la liste des lieux.

## Connexion à son compte GoogleDrive:

- Lancer la cellule
- Cliquer sur le lien généré en dessous
Copier le mot de passe dans la nouvelle fenêtre ouverte où l'accès au compte est requis et le coller en dessous de la cellule.

In [5]:
!pip install geopy
!pip install --upgrade wptools
from google.colab import drive
import folium
from geopy.geocoders import Nominatim
import pandas as pd
from folium import plugins
import json
import urllib.parse, urllib.request
import numpy as np
import csv
from collections import Counter
from branca.element import Figure
import wptools
import re
import itertools

drive.mount('/content/drive/')
%cd /content/drive/My Drive/NER-demo-lieux/

Requirement already up-to-date: wptools in /usr/local/lib/python3.7/dist-packages (0.4.17)
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/NER-demo-lieux


## Récupération des identifiants WIKIDATA des lieux

- Indiquer le chemin du fichier .txt avec la liste des lieux (fin de la cellule)
+ possibilité de changer le nom du fichier de sortie (fichier .txt avec liste des identifiants)
- Changer la langue si nécessaire (par défaut: français)
- Si la détéction n'est pas assez précis, descendre "thresold" à 0.8 (moins de noms vont être identifiés), dans la cas contraire, monter le seuil à 1. Par défaut, il est placé à une valeur moyenne de 0.9

In [2]:
def CallWikifier(text, lang="fr", threshold=1): 
  # Thresold = taux de sureté. 0.8 = sûr que le résultat est exact; mais peu de résultat.
  # 1 = tous les résultats mais certains faux résultats
    # Url de demande:
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "4"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
    # Appel de Wikifier 
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 100) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
        print(response)
    # Sortie des annotations:
    with open('ID-lieux1.txt', 'w') as outfile:
        for annotation in response["annotations"]:
          ## Récupération (uniquement de l'ID wikidata):
          itemid = annotation["wikiDataItemId"] 
          itemid2 = "".join([str(itemid)," "])
          outfile.write(itemid2)

with open ("chapitre1.txt", "r") as myfile:
    liste=myfile.read()
CallWikifier(text=liste, lang="fr")

{'annotations': [{'title': 'Place (voie)', 'url': 'http://fr.wikipedia.org/wiki/Place_(voie)', 'lang': 'fr', 'pageRank': 0.01947365741029875, 'secLang': 'en', 'secTitle': 'Town square', 'secUrl': 'http://en.wikipedia.org/wiki/Town_square', 'wikiDataItemId': 'Q174782', 'wikiDataClasses': [], 'dbPediaTypes': [], 'dbPediaIri': 'http://dbpedia.org/resource/Town_square', 'supportLen': 3, 'support': [{'wFrom': 0, 'wTo': 0, 'chFrom': 0, 'chTo': 4, 'pMentionGivenSurface': 0.01711017953157751, 'pageRank': 0.0007036677653253496, 'prbConfidence': 0.9281656062833974, 'entropy': 0.05889157293723925}, {'wFrom': 10, 'wTo': 10, 'chFrom': 61, 'chTo': 65, 'pMentionGivenSurface': 0.01711017953157751, 'pageRank': 0.0007036677653253496, 'prbConfidence': 0.9281656062833974, 'entropy': 0.05889157293723925}, {'wFrom': 17, 'wTo': 17, 'chFrom': 105, 'chTo': 109, 'pMentionGivenSurface': 0.01711017953157751, 'pageRank': 0.0007036677653253496, 'prbConfidence': 0.9281656062833974, 'entropy': 0.05889157293723925}]},

In [3]:
def CallWikifier(text, lang="fr", threshold=1): 
  # Thresold = taux de sureté. 0.8 = sûr que le résultat est exact; mais peu de résultat.
  # 1 = tous les résultats mais certains faux résultats
    # Url de demande:
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "4"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
    # Appel de Wikifier 
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 100) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
        print(response)
    # Sortie des annotations:
    with open('ID-lieux2.txt', 'w') as outfile:
        for annotation in response["annotations"]:
          ## Récupération (uniquement de l'ID wikidata):
          itemid = annotation["wikiDataItemId"] 
          itemid2 = "".join([str(itemid)," "])
          outfile.write(itemid2)

with open ("chapitre2.txt", "r") as myfile2:
    liste=myfile2.read()
CallWikifier(text=liste, lang="fr")

{'annotations': [{'title': 'Musée', 'url': 'http://fr.wikipedia.org/wiki/Mus%c3%a9e', 'lang': 'fr', 'pageRank': 0.01912862230105904, 'secLang': 'en', 'secTitle': 'Museum', 'secUrl': 'http://en.wikipedia.org/wiki/Museum', 'wikiDataItemId': 'Q33506', 'wikiDataClasses': [], 'dbPediaTypes': [], 'dbPediaIri': 'http://dbpedia.org/resource/Museum', 'supportLen': 1, 'support': [{'wFrom': 0, 'wTo': 0, 'chFrom': 0, 'chTo': 4, 'pMentionGivenSurface': 0.005289296861786368, 'pageRank': 0.0003236331900473656, 'prbConfidence': 0.3727405054626836, 'entropy': 0.8042560638212934}]}, {'title': 'Paris', 'url': 'http://fr.wikipedia.org/wiki/Paris', 'lang': 'fr', 'pageRank': 0.02423920457497429, 'secLang': 'en', 'secTitle': 'Paris', 'secUrl': 'http://en.wikipedia.org/wiki/Paris', 'wikiDataItemId': 'Q90', 'wikiDataClasses': [{'itemId': 'Q5119', 'enLabel': 'capital'}, {'itemId': 'Q484170', 'enLabel': 'commune of France'}, {'itemId': 'Q6465', 'enLabel': 'department of France'}, {'itemId': 'Q866196', 'enLabel':

## Récupération des informations de chaque lieu reconnu et lié à wikidata:

- Lancer la cellule

Toutes les informations (le type de lieu, les coordonnées GPS (latitude et longitude) ainsi qu'une brève description du lieu) sont enregistrées et exportées sous la forme d'un fichier de données JSON ainsi que sous la forme d'un tableau dans un fichier Excel dans le Google Drive. Il est possbile de changer le nom du fichier xlsx créé à la toute dernière ligne de la cellule: ici, il se nomme "informations_lieux.xlsx"

In [9]:
%cd /content/drive/My Drive/NER-demo-lieux/

donnees ={}
nom = []
type_lieu = []
latitude = []
longitude = []
longitude = []
description = []
# Utilisation du document créé précédement avec les identifiants wikidata:
with open("/content/drive/My Drive/NER-demo-lieux/ID-lieux1.txt", "r") as a_file:
      with open("donnees_lieux1.json", "w") as fichier : 
        for line in a_file:
            for word in line.split():
              page = wptools.page(wikibase=word, lang="fr")
              page.get_wikidata()
              page.get_parse()
## Pour avoir toutes les informations:
              try:
                infobox=page.data['infobox']
                nom = infobox.get("nom")
                type_lieu=infobox.get("type")
                if type_lieu is None: 
                  try:
                    type_lieu=page.data['what']
                  except KeyError as ke:
                    pass
                latitude = infobox.get("latitude")
                longitude = infobox.get("longitude")
                description = page.data["description"]
              except AttributeError as at:
                print("Ce n'est pas un lieu!")
# Création d'un dictionnaire + fichier json avec les informations récupérées sur les lieux:
              try:
                donnees[nom] = {"Type de lieu" : type_lieu,
                              "Latitude" : latitude, "Longitude" : longitude, "Description": description}
              except TypeError as tpy:
                pass
      
        json.dump(donnees, fichier, ensure_ascii=False)

# Enregistrement de toutes les informations récupérées sous la forme d'un tableau Excel:
df = pd.DataFrame(donnees)
df2 = df.transpose()
df2.to_excel('informations_lieux.xlsx')

/content/drive/My Drive/NER-demo-lieux


www.wikidata.org (wikidata) Q174782
www.wikidata.org (labels) P1424|P3827|P6771|P227|P1343|Q98929991|...
Note: Wikidata item Q174782 missing 'instance of' (P31)
fr.wikipedia.org (imageinfo) File:Московский Кремль №5.JPG
Place (voie) (fr) data
{
  claims: <dict(29)> P508, P373, P279, P349, P646, P1051, P1709, P...
  description: espace public non bâti, desservi par des voies
  image: <list(1)> {'file': 'File:Московский Кремль №5.JPG', 'kind...
  label: place
  labels: <dict(38)> P1424, P3827, P6771, P227, P1343, Q98929991, ...
  modified: <dict(1)> wikidata
  requests: <list(3)> wikidata, labels, imageinfo
  title: Place_(voie)
  wikibase: Q174782
  wikidata: <dict(27)> identifiant Bibliothèque nationale centrale...
  wikidata_pageid: 174672
  wikidata_url: https://www.wikidata.org/wiki/Q174782
}
fr.wikipedia.org (parse) Place_(voie)
Place (voie) (fr) data
{
  claims: <dict(29)> P508, P373, P279, P349, P646, P1051, P1709, P...
  description: espace public non bâti, desservi par des voie

Ce n'est pas un lieu!


www.wikidata.org (labels) P935|P6773|Q212793|P7822|Q1524643|Q2105...
www.wikidata.org (labels) Q4173137|P1465|Q3164528|Q22927616|Q2537...
www.wikidata.org (labels) P214|P30|Q207162|P94|P4173|P3608|P1791|...
www.wikidata.org (labels) Q369234|Q18086668|P443|P213|Q2727598|Q5...
www.wikidata.org (labels) P1968|Q3262481|P1225|P1997|Q146246|Q212...
www.wikidata.org (labels) Q2863958|Q15921401|Q12761|Q3212678|P691...
www.wikidata.org (labels) P1830|Q64436|P1667|Q612|Q5460604|Q17168...
www.wikidata.org (labels) Q5626403|Q3485987|Q88521107|Q201982|P13...
fr.wikipedia.org (imageinfo) File:Paris - Eiffelturm und Marsfeld2.jpg
Paris (fr) data
{
  aliases: <list(5)> Ville-Lumière, Paname, Lutèce, Ville de l'Amo...
  claims: <dict(183)> P2924, P1082, P1667, P1151, P1333, P349, P21...
  description: capitale de la France
  image: <list(1)> {'file': 'File:Paris - Eiffelturm und Marsfeld2...
  label: Paris
  labels: <dict(381)> P935, P6773, Q212793, P7822, Q1524643, Q2105...
  modified: <dict(1)> wikid

Ce n'est pas un lieu!


www.wikidata.org (labels) P3827|P8408|P1417|P5844|Q2604618|P279|P...
Note: Wikidata item Q1572168 missing 'instance of' (P31)
Moulinet (pêche) (fr) data
{
  aliases: <list(1)> moulinet de pêche
  claims: <dict(9)> P373, P646, P3827, P1417, P5844, P279, P8408, ...
  label: moulinet
  labels: <dict(11)> P3827, P8408, P1417, P5844, Q2604618, P279, P...
  modified: <dict(1)> wikidata
  requests: <list(2)> wikidata, labels
  title: Moulinet_(pêche)
  wikibase: Q1572168
  wikidata: <dict(9)> catégorie Commons (P373), identifiant Freeba...
  wikidata_pageid: 1508106
  wikidata_url: https://www.wikidata.org/wiki/Q1572168
}
fr.wikipedia.org (parse) Moulinet_(pêche)
Moulinet (pêche) (fr) data
{
  aliases: <list(1)> moulinet de pêche
  claims: <dict(9)> P373, P646, P3827, P1417, P5844, P279, P8408, ...
  label: moulinet
  labels: <dict(11)> P3827, P8408, P1417, P5844, Q2604618, P279, P...
  modified: <dict(1)> wikidata
  pageid: 1039612
  parsetree: <str(5126)> <root><template><title>Voir homonym

Ce n'est pas un lieu!


www.wikidata.org (labels) P454|P3222|P793|Q3447125|Q142|P31|Q1038...
fr.wikipedia.org (imageinfo) File:Manufacture des Gobelins.jpg
Manufacture des Gobelins (fr) data
{
  claims: <dict(20)> P31, P646, P380, P131, P669, P17, P18, P1435,...
  description: bâtiment dans le 13e arrondissement de Paris
  image: <list(1)> {'file': 'File:Manufacture des Gobelins.jpg', '...
  label: manufacture des Gobelins
  labels: <dict(31)> P454, P3222, P793, Q3447125, Q142, P31, Q1038...
  modified: <dict(1)> wikidata
  requests: <list(3)> wikidata, labels, imageinfo
  title: Manufacture_des_Gobelins
  what: bâtiment historique
  wikibase: Q83437773
  wikidata: <dict(20)> nature de l'élément (P31), identifiant Free...
  wikidata_pageid: 82754512
  wikidata_url: https://www.wikidata.org/wiki/Q83437773
}
fr.wikipedia.org (parse) Manufacture_des_Gobelins
Manufacture des Gobelins (fr) data
{
  claims: <dict(20)> P31, P646, P380, P131, P669, P17, P18, P1435,...
  description: bâtiment dans le 13e arrondissemen

In [28]:
%cd /content/drive/My Drive/NER-demo-lieux/

donnees ={}
nom = []
type_lieu = []
latitude = []
longitude = []
longitude = []
description = []
# Utilisation du document créé précédement avec les identifiants wikidata:
with open("/content/drive/My Drive/NER-demo-lieux/ID-lieux2.txt", "r") as a_file:
      with open("donnees_lieux2.json", "w") as fichier : 
        for line in a_file:
            for word in line.split():
              page = wptools.page(wikibase=word, lang="fr")
              page.get_wikidata()
              page.get_parse()
## Pour avoir toutes les informations:
              try:
                infobox=page.data['infobox']
                nom = infobox.get("nom")
                type_lieu=infobox.get("type")
                if type_lieu is None: 
                  try:
                    type_lieu=page.data['what']
                  except KeyError as ke:
                    pass
                latitude = infobox.get("latitude")
                longitude = infobox.get("longitude")
                description = page.data["description"]
              except AttributeError as at:
                print("Ce n'est pas un lieu!")
# Création d'un dictionnaire + fichier json avec les informations récupérées sur les lieux:
              try:
                donnees[nom] = {"Type de lieu" : type_lieu,
                              "Latitude" : latitude, "Longitude" : longitude, "Description": description}
              except TypeError as tpy:
                pass
      
        json.dump(donnees, fichier, ensure_ascii=False)

# Enregistrement de toutes les informations récupérées sous la forme d'un tableau Excel:
df = pd.DataFrame(donnees)
df2 = df.transpose()
df2.to_excel('informations_lieux2.xlsx')

/content/drive/My Drive/NER-demo-lieux


www.wikidata.org (wikidata) Q33506
www.wikidata.org (labels) P935|P486|Q190804|P2283|P7822|Q10389221...
www.wikidata.org (labels) P7033|P1807|P3916|Q17334923|Q2668072|P6...
Note: Wikidata item Q33506 missing 'instance of' (P31)
fr.wikipedia.org (imageinfo) File:GD-EG-Caire-Musée007.JPG
Musée (fr) data
{
  aliases: <list(3)> centres d'interprétation, musee, musées
  claims: <dict(77)> P1282, P373, P279, P508, P910, P349, P227, P6...
  description: <str(98)> institution qui conserve, collecte et exp...
  image: <list(1)> {'file': 'File:GD-EG-Caire-Musée007.JPG', 'kind...
  label: musée
  labels: <dict(97)> P935, P486, Q190804, P2283, P7822, Q103892210...
  modified: <dict(1)> wikidata
  requests: <list(4)> wikidata, labels, labels, imageinfo
  title: Musée
  wikibase: Q33506
  wikidata: <dict(76)> attribut ou clé OpenStreetMap (P1282), caté...
  wikidata_pageid: 36344
  wikidata_url: https://www.wikidata.org/wiki/Q33506
}
fr.wikipedia.org (parse) Musée
Musée (fr) data
{
  aliases: <list(

Ce n'est pas un lieu!


www.wikidata.org (labels) P935|P6773|Q212793|P7822|Q1524643|Q2105...
www.wikidata.org (labels) Q4173137|P1465|Q3164528|Q22927616|Q2537...
www.wikidata.org (labels) P214|P30|Q207162|P94|P4173|P3608|P1791|...
www.wikidata.org (labels) Q369234|Q18086668|P443|P213|Q2727598|Q5...
www.wikidata.org (labels) P1968|Q3262481|P1225|P1997|Q146246|Q212...
www.wikidata.org (labels) Q2863958|Q15921401|Q12761|Q3212678|P691...
www.wikidata.org (labels) P1830|Q64436|P1667|Q612|Q5460604|Q17168...
www.wikidata.org (labels) Q5626403|Q3485987|Q88521107|Q201982|P13...
fr.wikipedia.org (imageinfo) File:Paris - Eiffelturm und Marsfeld2.jpg
Paris (fr) data
{
  aliases: <list(5)> Ville-Lumière, Paname, Lutèce, Ville de l'Amo...
  claims: <dict(183)> P2924, P1082, P1667, P1151, P1333, P349, P21...
  description: capitale de la France
  image: <list(1)> {'file': 'File:Paris - Eiffelturm und Marsfeld2...
  label: Paris
  labels: <dict(381)> P935, P6773, Q212793, P7822, Q1524643, Q2105...
  modified: <dict(1)> wikid

Ce n'est pas un lieu!


...

www.wikidata.org (labels) Q193899|P421|Q142|P214|Q9220348|Q18102|...
www.wikidata.org (labels) P1465|Q7909591|Q8048671|P6671|Q84114824...
fr.wikipedia.org (imageinfo) File:Château de Vincennes Paris FRA ...
Vincennes (fr) data
{
  claims: <dict(59)> P1036, P190, P910, P373, P374, P17, P94, P625...
  description: commune française du département du Val-de-Marne
  image: <list(1)> {'file': 'File:Château de Vincennes Paris FRA 0...
  label: Vincennes
  labels: <dict(86)> Q193899, P421, Q142, P214, Q9220348, Q18102, ...
  modified: <dict(1)> wikidata
  requests: <list(4)> wikidata, labels, labels, imageinfo
  title: Vincennes
  what: commune de France
  wikibase: Q193819
  wikidata: <dict(57)> classification décimale de Dewey (P1036), j...
  wikidata_pageid: 191794
  wikidata_url: https://www.wikidata.org/wiki/Q193819
}
fr.wikipedia.org (parse) Vincennes
fr.wikipedia.org (imageinfo) File:Donjon du Château de Vincennes.jpg
Vincennes (fr) data
{
  claims: <dict(59)> P1036, P190, P910, P373, 

Ce n'est pas un lieu!


www.wikidata.org (labels) P1424|P9230|P3221|Q7961238|P3827|P2167|...
www.wikidata.org (labels) P910|P2924|P5604|P5250|P1245|P646|P155|...
Note: Wikidata item Q43501 missing 'instance of' (P31)
fr.wikipedia.org (imageinfo) File:Przewalskis horses exposition, ...
Parc zoologique (fr) data
{
  aliases: <list(3)> zoo, jardin zoologique, Zoo jardin
  claims: <dict(57)> P1282, P373, P508, P279, P910, P349, P227, P6...
  description: espace où sont réunies de nombreuses espèces animales
  image: <list(1)> {'file': 'File:Przewalskis horses exposition, Z...
  label: parc zoologique
  labels: <dict(68)> P1424, P9230, P3221, Q7961238, P3827, P2167, ...
  modified: <dict(1)> wikidata
  requests: <list(4)> wikidata, labels, labels, imageinfo
  title: Parc_zoologique
  wikibase: Q43501
  wikidata: <dict(56)> attribut ou clé OpenStreetMap (P1282), caté...
  wikidata_pageid: 45694
  wikidata_url: https://www.wikidata.org/wiki/Q43501
}
fr.wikipedia.org (parse) Parc_zoologique


Ce n'est pas un lieu!


Parc zoologique (fr) data
{
  aliases: <list(3)> zoo, jardin zoologique, Zoo jardin
  claims: <dict(57)> P1282, P373, P508, P279, P910, P349, P227, P6...
  description: espace où sont réunies de nombreuses espèces animales
  image: <list(1)> {'file': 'File:Przewalskis horses exposition, Z...
  iwlinks: <list(12)> https://en.wikipedia.org/wiki/Greek_gardens,...
  label: parc zoologique
  labels: <dict(68)> P1424, P9230, P3221, Q7961238, P3827, P2167, ...
  modified: <dict(1)> wikidata
  pageid: 114061
  parsetree: <str(55336)> <root><template><title>confusion</title>...
  requests: <list(5)> wikidata, labels, labels, imageinfo, parse
  title: Parc zoologique
  wikibase: Q43501
  wikidata: <dict(56)> attribut ou clé OpenStreetMap (P1282), caté...
  wikidata_pageid: 45694
  wikidata_url: https://www.wikidata.org/wiki/Q43501
  wikitext: <str(45315)> {{confusion|Parc animalier}}{{Voir homony...
}


##Création d'une carte pour comparer deux parcours

In [29]:

keys_lat = donnees[nom]["Latitude"]
keys_long = donnees[nom]["Longitude"]
liste_des_lat = []
liste_des_long = []
fichier_json = "/content/drive/My Drive/NER-demo-lieux/donnees_lieux1.json"
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  print(pairs)
  for key, value in pairs:
      test_lat=value["Latitude"]
      liste_des_lat.append(test_lat)
      test_long=value["Longitude"]
      liste_des_long.append(test_long)

coords = []
for ju, ij in zip(liste_des_lat, liste_des_long):
  if ju is None:
    pass  
  elif ij is None:
    pass
  else:
    coords.append([float(ju), float(ij)])
#print(coords) ### Crée un array de coordonnées en suppirimant les None


keys_lat2 = donnees2[nom2]["Latitude"]
keys_long2 = donnees2[nom2]["Longitude"]
liste_des_lat2 = []
liste_des_long2 = []
fichier_json2 = "/content/drive/My Drive/NER-demo-lieux/donnees_lieux2.json"
with open(fichier_json2, "r") as f2 :
  contenu_json2 = json.load(f2)
  pairs2 = contenu_json2.items()
  for key2, value2 in pairs2:
      test_lat2=value2["Latitude"]
      liste_des_lat2.append(test_lat2)
      test_long2=value2["Longitude"]
      liste_des_long2.append(test_long2)
coords2 = []
for ju2, ij2 in zip(liste_des_lat2, liste_des_long2):
  if ju2 is None:
    pass  
  elif ij2 is None:
    pass
  else:
    coords2.append([float(ju2), float(ij2)])
print(coords2) ### Crée un array de coordonnées en suppirimant les None

fig5=Figure(height=550,width=750)
m5=folium.Map(location=[48.856614, 2.3522219], zoom_start=12)
fig5.add_child(m5)

f1=folium.FeatureGroup("Parcours1")
f2=folium.FeatureGroup("Parcours2")
line_1=folium.vector_layers.PolyLine(coords,popup='<b>Parcours_test</b>',tooltip='Tour de France',color='blue',weight=4).add_to(f1)
line_2=folium.vector_layers.PolyLine(coords2,popup='<b>Parcours_test2</b>',tooltip='Tour de France2',color='red',weight=4).add_to(f2)
f1.add_to(m5)
f2.add_to(m5)
folium.LayerControl().add_to(m5)



m5

dict_items([('Paris', {'Type de lieu': 'capitale', 'Latitude': '48.856578', 'Longitude': '2.351828', 'Description': 'capitale de la France'}), ('Madeleine', {'Type de lieu': None, 'Latitude': None, 'Longitude': None, 'Description': 'pâtisserie'}), ('null', {'Type de lieu': 'bâtiment historique', 'Latitude': None, 'Longitude': None, 'Description': 'bâtiment dans le 13e arrondissement de Paris'}), ("Place d'Italie", {'Type de lieu': 'station de métro', 'Latitude': '48.831483', 'Longitude': '2.355692', 'Description': 'station du métro de Paris'}), ("Avenue d'Italie", {'Type de lieu': 'avenue', 'Latitude': '48.825505', 'Longitude': '2.3574726', 'Description': 'principale artère du 13e arrondissement de Paris'})])
[[48.856578, 2.351828], [48.853333, 2.369167], [48.8477777778, 2.43916666667]]


## Créer et ajouter son fond de carte

Si l'on veut ajouter son PROPRE fond de carte personnalisé:

- Création d’un compte sur : https://mapwarper.net/

- Téléchargement du fond de carte (IIIF recommandé, avec la meilleure résolution possible)

- Sur Map Warper: dans l’onglet « Upload Map » il est possible de charger le fond de carte que l’on veut utiliser, et indiquer toutes les métadonnées nécessaires.

- Dans l’onglet "Rectify": il faut indiquer le plus de points identiques sur les deux cartes pour permettre la superposition du fond de carte chargé sur des données géolocalisées. Cliquer sur un point sur chaque carte, puis cliquer sur « add Control point » en dessous des cartes. Répéter l’opération le plus de fois possible pour obtenir un bon résultat. Dans l'encadré en dessous des deux fonds de carte, cliquer sur "Warp image": le fond s'ajoute. Si le résultat n'est pas satisfaisant, il possbile d'ajouter d'autres repères pour améliorer la qualité du nouveau fond de carte créé. 

- Se rendre dans l'onglet "Export". Dans "Map services", copier le lien entier se situant après "Tiles (Google/OSM scheme):", débutant par http et se terminant par .png

- Coller ensuite le lien à la troisième ligne de la cellule suivante, à la place du lien existant, après "tiles = " en conservant bien les guillemets.

In [30]:
fig5=Figure(height=550,width=750)
m5=folium.Map(location=[48.856614, 2.3522219], zoom_start=12,
                 tiles="https://mapwarper.net/maps/tile/54525/{z}/{x}/{y}.png", attr="None")



keys_lat = donnees[nom]["Latitude"]
keys_long = donnees[nom]["Longitude"]
liste_des_lat = []
liste_des_long = []
fichier_json = "/content/drive/My Drive/NER-demo-lieux/donnees_lieux1.json"
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  print(pairs)
  for key, value in pairs:
      test_lat=value["Latitude"]
      liste_des_lat.append(test_lat)
      test_long=value["Longitude"]
      liste_des_long.append(test_long)

coords = []
for ju, ij in zip(liste_des_lat, liste_des_long):
  if ju is None:
    pass  
  elif ij is None:
    pass
  else:
    coords.append([float(ju), float(ij)])
#print(coords) ### Crée un array de coordonnées en suppirimant les None


keys_lat2 = donnees2[nom2]["Latitude"]
keys_long2 = donnees2[nom2]["Longitude"]
liste_des_lat2 = []
liste_des_long2 = []
fichier_json2 = "/content/drive/My Drive/NER-demo-lieux/donnees_lieux2.json"
with open(fichier_json2, "r") as f2 :
  contenu_json2 = json.load(f2)
  pairs2 = contenu_json2.items()
  for key2, value2 in pairs2:
      test_lat2=value2["Latitude"]
      liste_des_lat2.append(test_lat2)
      test_long2=value2["Longitude"]
      liste_des_long2.append(test_long2)
coords2 = []
for ju2, ij2 in zip(liste_des_lat2, liste_des_long2):
  if ju2 is None:
    pass  
  elif ij2 is None:
    pass
  else:
    coords2.append([float(ju2), float(ij2)])
print(coords2) ### Crée un array de coordonnées en suppirimant les None

f1=folium.FeatureGroup("Parcours1")
f2=folium.FeatureGroup("Parcours2")
line_1=folium.vector_layers.PolyLine(coords,popup='<b>Parcours_test</b>',tooltip='Tour de France',color='blue',weight=4).add_to(f1)
line_2=folium.vector_layers.PolyLine(coords2,popup='<b>Parcours_test2</b>',tooltip='Tour de France2',color='red',weight=4).add_to(f2)
f1.add_to(m5)
f2.add_to(m5)
folium.LayerControl().add_to(m5)



m5


dict_items([('Paris', {'Type de lieu': 'capitale', 'Latitude': '48.856578', 'Longitude': '2.351828', 'Description': 'capitale de la France'}), ('Madeleine', {'Type de lieu': None, 'Latitude': None, 'Longitude': None, 'Description': 'pâtisserie'}), ('null', {'Type de lieu': 'bâtiment historique', 'Latitude': None, 'Longitude': None, 'Description': 'bâtiment dans le 13e arrondissement de Paris'}), ("Place d'Italie", {'Type de lieu': 'station de métro', 'Latitude': '48.831483', 'Longitude': '2.355692', 'Description': 'station du métro de Paris'}), ("Avenue d'Italie", {'Type de lieu': 'avenue', 'Latitude': '48.825505', 'Longitude': '2.3574726', 'Description': 'principale artère du 13e arrondissement de Paris'})])
[[48.856578, 2.351828], [48.853333, 2.369167], [48.8477777778, 2.43916666667]]
